In [1]:
import pandas as pd
import numpy as np
import folium as f
import matplotlib.pyplot as plt #画图工具
import matplotlib.image as mpimg # mpimg 用于读取图片

In [4]:
# 轨道站点进站客流数据
!wget --no-check-certificate https://download11.ufile.ucloud.com.cn/competition-file/gdzdtjsj_jzkl.rar
# 轨道站点出站客流数据
!wget --no-check-certificate https://download11.ufile.ucloud.com.cn/competition-file/gdzdtjsj_czkl.rar
# 轨道站点闸机设备编码数据
!wget --no-check-certificate https://download11.ufile.ucloud.com.cn/competition-file/gdzdkltj_zjbh.rar

--2021-02-22 12:37:58--  https://download11.ufile.ucloud.com.cn/competition-file/gdzdtjsj_jzkl.rar
Resolving download11.ufile.ucloud.com.cn (download11.ufile.ucloud.com.cn)... 111.32.171.50, 111.32.171.10, 111.32.171.49, ...
Connecting to download11.ufile.ucloud.com.cn (download11.ufile.ucloud.com.cn)|111.32.171.50|:443... connected.
	requested host name 'download11.ufile.ucloud.com.cn'.
HTTP request sent, awaiting response... 200 OK
Length: 139070 (136K) [application/octet-stream]
Saving to: 'gdzdtjsj_jzkl.rar'

     0K .......... .......... .......... .......... .......... 36% 1.31M 0s
    50K .......... .......... .......... .......... .......... 73% 6.60M 0s
   100K .......... .......... .......... .....                100% 5.33M=0.05s

2021-02-22 12:37:58 (2.59 MB/s) - 'gdzdtjsj_jzkl.rar' saved [139070/139070]

--2021-02-22 12:37:59--  https://download11.ufile.ucloud.com.cn/competition-file/gdzdtjsj_czkl.rar
Resolving download11.ufile.ucloud.com.cn (download11.ufile.ucloud.com.cn)

In [13]:
# ./当前目录 ../上级目录
data_path = './data/'
save_path = './result/'

In [27]:
# 轨道站点闸机设备编码数据
zjbh = pd.read_excel(data_path + 'gdzdkltj_zjbh.csv')
zjbh.info()
# 轨道站点出站客流数据
czkl = pd.read_excel('./data/gdzdtjsj_czkl.csv') 
czkl.describe
#轨道站点进站客流数据
jzkl = pd.read_excel('./data/gdzdtjsj_jzkl.csv')
jzkl.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   线路号                 5 non-null      object 
 1   站点编号                5 non-null      float64
 2   站点名称                5 non-null      object 
 3   闸机编号（RAIL_GATE_ID）  6 non-null      object 
 4   Unnamed: 4          6 non-null      object 
 5   Unnamed: 5          6 non-null      object 
 6   Unnamed: 6          6 non-null      object 
dtypes: float64(1), object(6)
memory usage: 464.0+ bytes


,STATISTICAL_TIME,STATION_NO,RAIL_GATE_ID,NO_PASSENGERS
0,统计时间,轨道站点编号,闸机编号,客流量
1,2020-12-20 06:00:00,0110,10c9ae,1
2,2020-12-20 06:09:59.999996,0108,107b89,2
3,2020-12-20 06:09:59.999996,0108,107b8a,3
4,2020-12-20 06:09:59.999996,0108,107b8b,1
...,...,...,...,...
95,2020-12-20 07:09:59.999999,0219,216b85,1
96,2020-12-20 07:09:59.999999,0219,216b86,1
97,2020-12-20 07:09:59.999999,0219,216b87,1
98,2020-12-20 07:09:59.999999,0219,216b8d,10


In [29]:
# 共享单车轨迹数据
bike_track = pd.concat([
    pd.read_csv(data_path + 'gxdc_gj20201221.csv'),
    pd.read_csv(data_path + 'gxdc_gj20201222.csv'),
    pd.read_csv(data_path + 'gxdc_gj20201223.csv'),
    pd.read_csv(data_path + 'gxdc_gj20201224.csv'),
    pd.read_csv(data_path + 'gxdc_gj20201225.csv')
])

# 按照单车ID和时间进行排序
bike_track = bike_track.sort_values(['BICYCLE_ID', 'LOCATING_TIME'])

In [30]:
def bike_fence_format(s):
    s = s.replace('[', '').replace(']', '').split(',')
    s = np.array(s).astype(float).reshape(5, -1)
    return s

# 共享单车停车点位（电子围栏）数据
bike_fence = pd.read_csv(data_path + 'gxdc_tcd.csv')
bike_fence['FENCE_LOC'] = bike_fence['FENCE_LOC'].apply(bike_fence_format)

In [31]:
# 共享单车订单数据
bike_order = pd.read_csv(data_path + 'gxdc_dd.csv')
bike_order = bike_order.sort_values(['BICYCLE_ID', 'UPDATE_TIME'])

In [32]:
# 首先我们可以统计不同品牌单车的利用率：
bike_use_ratio = bike_track.groupby(['source','BICYCLE_ID', 'hour'])['date'].count() * 15.0 / 3615
bike_use_ratio = bike_use_ratio.reset_index()
bike_use_ratio.groupby(['source', 'hour'])['date'].mean()

source   hour
halou    6       0.157031
         7       0.221240
         8       0.315490
         9       0.223382
meituan  6       0.092250
         7       0.099389
         8       0.099104
         9       0.108867
qingju   6       0.054113
         7       0.161101
         8       0.195099
         9       0.120881
Name: date, dtype: float64

In [ ]:
# 根据上一章节的街道潮汐统计，我们可以统计得出每个街道的具体潮汐情况，并进而可以按照密度计算得到潮汐情况最为严重的区域。
# 为了对厦门市潮汐情况与区域关系进行可视化，来对具体的潮汐情况进行直观的感受。首先我们按照之前的代码完成具体数据读取，
# 并对单车订单数据的时间进行提取：为了方便统计这里，我们只使用一个时间的订单数据进行绘制：

In [33]:
bike_order['UPDATE_TIME'] = pd.to_datetime(bike_order['UPDATE_TIME'])
bike_order['DAY'] = bike_order['UPDATE_TIME'].dt.day.astype(object)
bike_order['DAY'] = bike_order['DAY'].apply(str)

bike_order['HOUR'] = bike_order['UPDATE_TIME'].dt.hour.astype(object)
bike_order['HOUR'] = bike_order['HOUR'].apply(str)
bike_order['HOUR'] = bike_order['HOUR'].str.pad(width=2,side='left',fillchar='0')

bike_order['DAY_HOUR'] = bike_order['DAY'] + bike_order['HOUR']

In [37]:
import folium
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[24.482426, 118.157606], zoom_start=14)
HeatMap(bike_order.loc[(bike_order['DAY_HOUR'] == '2106') & (bike_order['LOCK_STATUS'] == 1), 
                   ['LATITUDE', 'LONGITUDE']]).add_to(map_hooray)
# for data in bike_fence['FENCE_LOC'].values[::10]:
#     folium.Marker(
#         data[0, ::-1]
#     ).add_to(map_hooray)
# map_hoora

bi = bike_fence['FENCE_LOC'].values[:10]
for row in bi:
    for fence_item in row:
        folium.Marker(
            location = [fence_item[1], fence_item[0]]
#             location = fence_item.reverse()
        ).add_to(map_hooray)
map_hooray  

In [ ]:
# 根据上一章节的街道潮汐统计，我们可以得到每个停车点具体的潮汐情况。如果此停车点停满了，进一步我们也可以推荐其他停车点。
# 我们使用下面的代码完成具体的停车点潮汐统计：

In [38]:
import geohash
from geopy.distance import geodesic
bike_order['geohash'] = bike_order.apply(lambda x: 
                        geohash.encode(x['LATITUDE'], x['LONGITUDE'], precision=9), axis=1)
bike_fence['MIN_LATITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.min(x[:, 1]))
bike_fence['MAX_LATITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.max(x[:, 1]))

bike_fence['MIN_LONGITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.min(x[:, 0]))
bike_fence['MAX_LONGITUDE'] = bike_fence['FENCE_LOC'].apply(lambda x: np.max(x[:, 0]))

bike_fence['FENCE_AREA'] = bike_fence.apply(lambda x: geodesic(
    (x['MIN_LATITUDE'], x['MIN_LONGITUDE']), (x['MAX_LATITUDE'], x['MAX_LONGITUDE'])
).meters, axis=1)

bike_fence['FENCE_CENTER'] = bike_fence['FENCE_LOC'].apply(
    lambda x: np.mean(x[:-1, ::-1], 0)
)
bike_order['geohash'] = bike_order.apply(
    lambda x: geohash.encode(x['LATITUDE'], x['LONGITUDE'], precision=7), 
axis=1)

bike_fence['geohash'] = bike_fence['FENCE_CENTER'].apply(
    lambda x: geohash.encode(x[0], x[1], precision=7)
)

bike_inflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 1], 
                   values='LOCK_STATUS', index=['geohash'],
                    columns=['DAY'], aggfunc='count', fill_value=0
)

bike_outflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 0], 
                   values='LOCK_STATUS', index=['geohash'],
                    columns=['DAY'], aggfunc='count', fill_value=0
)

bike_remain = (bike_inflow - bike_outflow).fillna(0)
bike_remain[bike_remain < 0] = 0  
bike_remain = bike_remain.sum(1)
bike_fence['DENSITY'] = bike_fence['geohash'].map(bike_remain).fillna(0)

In [39]:
# 如果乘客到了一个潮汐停车点A，如何对乘客进行引导呢？
# 推荐的停车点B应该和A距离不远；推荐的停车点B应该没有潮汐情况；
# 也需要考虑到跨街道和横穿马路的情况；首先为了完成停车点距离计算，还是先定义好KNN：
# 首先为了完成停车点距离计算，还是先定义好KNN：

In [40]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric = "haversine", n_jobs=-1, algorithm='brute')
knn.fit(np.stack(bike_fence['FENCE_CENTER'].values))

NearestNeighbors(algorithm='brute', metric='haversine', n_jobs=-1)

In [41]:
# 然后是具体的计算逻辑进行编码：停车引导计划
# 此外还需要考虑到单车没有起到停车点的情况，即推荐根据经纬度推荐最近非潮汐停车点：

In [42]:
def fence_recommend1(fence_id):
    fence_center = bike_fence.loc[bike_fence['FENCE_ID']==fence_id, 'FENCE_CENTER'].values[0]
    # 具体街道方向
    fence_dir = fence_id.split('_')[1]
    # 根据距离计算最近的20个待选位置
    dist, index = knn.kneighbors([fence_center], n_neighbors=20)
    print(dist)
    print(index)
    # 对每个待选位置进行筛选
    for idx in index[0]:
        # 剔除已经有很多车的
        if bike_fence.iloc[idx]['DENSITY'] > 10:
            continue
        # 剔除需要过街的
        if fence_dir not in bike_fence.iloc[idx]['FENCE_ID']:
            continue
        return bike_fence.iloc[idx]['FENCE_ID']
    return None

In [45]:
# 需要11s左右
dist, index = knn.kneighbors(bike_order[['LATITUDE','LONGITUDE']].values[:], n_neighbors=1)

In [46]:
# 然后计算所有停车点的潮汐流量：

bike_order['fence'] = bike_fence.iloc[index.flatten()]['FENCE_ID'].values

bike_inflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 1], 
                   values='LOCK_STATUS', index=['fence'],
                    columns=['DAY'], aggfunc='count', fill_value=0
)

bike_outflow = pd.pivot_table(bike_order[bike_order['LOCK_STATUS'] == 0], 
                   values='LOCK_STATUS', index=['fence'],
                    columns=['DAY'], aggfunc='count', fill_value=0
)

bike_remain = (bike_inflow - bike_outflow).fillna(0)
bike_remain[bike_remain < 0] = 0  
bike_remain = bike_remain.sum(1)

In [58]:
# 计算停车点的密度：
bike_density = bike_remain / bike_fence.set_index('FENCE_ID')['FENCE_AREA']
bike_density = bike_density.sort_values(ascending=False).reset_index()
bike_density = bike_density.fillna(0)
top100 = bike_density.head(100)
print(top100.iloc[:,0])
top100.info()

0     观日路(望海路至会展路段 )_R_1
1               象屿路0_R_1
2               望海路0_R_2
3               望海路0_R_1
4             云顶北路0_R_45
             ...        
95        莲前东路0_R_A02006
96             曾厝垵西路_R_2
97        高崎南五路_R_B27002
98             云顶中路0_L_8
99              云顶北路_R_4
Name: index, Length: 100, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   100 non-null    object 
 1   0       100 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


In [63]:
fence_recommend1('望海路0_R_2')

'环岛干道0_R_A06010'

In [61]:
# 推荐潮汐top100最近20个附近站点
for raw in top100.iloc[:,0] :
    fence_recommend20 = fence_recommend1(raw)
    print(fence_recommend20)

环岛干道0_R_30
None
环岛干道0_R_A06010
环岛干道0_R_30
None
None
None
None
会展路(塔埔路至吕岭路段 )_L_3_A
龙伏路0_L_5
创新路(马垄路至火炬北路)_R_2
兴湖路_L_A17001
禾山路_L_1
育秀东路_L_A09001
湖滨西路0_L_12
金边路0_R_A04005
环岛干道0_R_A07002
会展北路0_R_3
None
None
火炬路0_L_6
None
华容路_R_5
None
None
None
林后路_L_1
云顶北路0_L_5
None
None
None
枋湖北路_L_A19002
None
枋钟路0_L_7
None
高崎道_R_4
None
湖滨南路0_L_10
火炬路_R_1
湖滨南路0_L_10
None
创新三路0_R_5
林后路_L_1
None
体育路(东岳二路至莲岳路)_L_11
岭下路_L_2
None
None
湖滨南路0_L_13
云顶北路_L_B27019
创新三路0_R_5
None
岐山北二路_R_5
None
None
安岭路_R_4
金尚路(吕岭路至仙岳路段)_R_12
仙岳路连接线0_R_A20018
创新三路0_R_5
None
江头东路_L_2
None
None
None
None
创新三路0_R_5
长兴路_L_7
None
曾厝垵北路(龙虎山路至曾厝垵西路段)_R_3
None
None
马垅路_L_1
None
岭下东路(安岭路至岭下路)_R_1
None
创新三路0_L_1
吕岭路_L_B10019
白鹭洲路0_R_11
仙岳路_L_1
林后路0_R_42
莲前东路_R_B10001
None
翔远三路_R_1
湖滨北路(嘉禾路与莲岳路段)_R_3
None
鳌山路0_L_1
民族路_R_8
湖滨北路(湖滨中路至湖滨东路)R12
大学路0_L_A13001
鳌山路0_L_1
吕岭路(环岛干道至环岛东路段)_L_4
云顶中路0_L_31
None
None
金山中路_R_28
莲前东路0_R_A02006
曾厝垵北路(龙虎山路至曾厝垵西路段)_R_3
None
None
None


In [ ]:
def fence_recommend2(la, lo):
    dist, index = knn.kneighbors([[la, lo]], n_neighbors=20)
    for idx in index[0][1:]:
        if bike_fence.iloc[idx]['DENSITY'] > 10:
            continue
        return bike_fence.iloc[idx]['FENCE_ID']
    return None